# Assignment 5 #
### Due: Friday, November 17th to be submitted via Canvas by 11:59 pm ###
### Total points: **65** ###

# Q1: Support Vector Machines (10 points)

In this question, we will explore support vector machines for the Spam Base dataset from the UCI repository.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
 #!pip install ucimlrepo

In [ ]:
# from ucimlrepo import fetch_ucirepo
# # fetch dataset
# spambase = fetch_ucirepo(id=94)

# # data (as pandas dataframes)
# X = spambase.data.features
# y = spambase.data.targets
# print("Abstract:", spambase.metadata['abstract'])
# print("Number of instances:", spambase.metadata['num_instances'])
# print("Number of features:", spambase.metadata['num_features'])
# print(spambase.variables['name'])

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
#from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
#uploaded = files.upload()

In [3]:
df = pd.read_csv("data/spambase/spambase.data", header=None)
y = df[57]
X = df.drop(columns=[57])

In [4]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [5]:
seed = 42
y = y.to_numpy().squeeze()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=seed)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

a. (5 points) Implement the following function to train SVMs with a specified kernel type, hyper-parameter search space, and random state on the Spam Base dataset. Do hyper-parameter search over $C$ using [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), setting the number of folds to 5. After finding the best C, please use it to train the final model and return both the final model and the best C.

In [6]:
def search_best_svm(kernel, C_search_space, random_state):
    best_score = -np.inf
    for C in C_search_space:

        # Initialize an SVM classifier with the specified kernel type, C value, and random state
        ### START CODE ###
        clf = SVC(kernel=kernel, C=C, random_state=random_state)
        ### END CODE ###

        # Evaluate accuracy scores using 5-fold cross-validation scores
        ### START CODE ###
        scores = cross_val_score(clf, X_train, y_train, cv=5)
        ### END CODE ###

        # Compute the average score and compare with the current best score to update the best C
        ### START CODE ###
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_C = C  
        ### END CODE ###
        print(f"C: {C} Avg Cross Val Score: {np.round(score, 4)}")

    print(f"Best C: {best_C}")

    # Initialize the model using the specified kernel type, best C, and random state;
    # and then fit the model using training set
    ### START CODE ###
    model = SVC(kernel=kernel, C=best_C, random_state=random_state)
    model.fit(X_train, y_train)
    ### END CODE ###
    return model, best_C

b. (3 points) Run the function you implemented above to train SVMs with the search space of $C$ being [$0.1, 1, 10, 100$], random state set to 42, with the following three popular kernels: (i) linear (ii) polynomial (iii) RBF (Gaussian). Evaluate your final models on the test set and report their accuracies.

In [8]:
search_space = [0.1, 1, 10, 100]
random_state = 42
models = {}

In [9]:
for kernel in ['linear', 'poly', 'rbf']:
    # Train the model using the search_best_svm function
    model, best_C = search_best_svm(kernel, search_space, random_state)

    # Store the model in the dictionary
    models[kernel] = model

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Kernel: {kernel}, Best C: {best_C}, Accuracy: {accuracy}")

C: 0.1 Avg Cross Val Score: 0.9214
C: 1 Avg Cross Val Score: 0.9264
C: 10 Avg Cross Val Score: 0.9283
C: 100 Avg Cross Val Score: 0.9268
Best C: 10
Kernel: linear, Best C: 10, Accuracy: 0.9282998370450842
C: 0.1 Avg Cross Val Score: 0.6895
C: 1 Avg Cross Val Score: 0.7594
C: 10 Avg Cross Val Score: 0.8428
C: 100 Avg Cross Val Score: 0.8957
Best C: 100
Kernel: poly, Best C: 100, Accuracy: 0.9125475285171103
C: 0.1 Avg Cross Val Score: 0.896
C: 1 Avg Cross Val Score: 0.9243
C: 10 Avg Cross Val Score: 0.9243
C: 100 Avg Cross Val Score: 0.9123
Best C: 10
Kernel: rbf, Best C: 10, Accuracy: 0.935361216730038


c. (2 points) Train a logistic regression model using the training set. Compare its performance with that of the SVMs trained above.

In [11]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy}")


Logistic Regression Accuracy: 0.9223248234655079


# Question 2 : Ensemble Methods for Classification (25 pts)

In this question, we will compare the performances of different ensemble methods for classification problems: [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) classifiers.

We will look at the [GiveMeSomeCredit](https://www.kaggle.com/c/GiveMeSomeCredit) dataset for this question. The dataset is extremely large so for this question we will only consider a subset which has been provided along with the notebook for this assignment. The dataset has already been split into train and test sets.

The task is to predict the probability that someone will experience financial distress in the next two years.

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
#from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
#uploaded = files.upload()

In [ ]:
import pandas as pd

data = pd.read_csv('hw5_data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
y = data['SeriousDlqin2yrs']
X = data.drop(['SeriousDlqin2yrs'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

print('train:',X_train.shape, y_train.shape)
print('test:',X_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from time import time
import xgboost
%matplotlib inline

In [ ]:
columns_list = list(X.columns)

a. (2.5 pts) Fit a Decision Tree Classifier with random_state = 14 for this classification problem. Report the accuracy_score and roc_auc_score on the test set.

In [ ]:
def fit_classifier(clf):
  # Fit the classifier on the training set
  ### START CODE ###
  ### END CODE ###
  return clf

In [ ]:
def evaluate_classifier(clf, X_test, y_test):
  # Compute the accuracy_score, and roc_auc_score on the test set
  ### START CODE ###
  y_pred = None
  y_pred_proba = None

  acc_score = accuracy_score(None, None)
  auc_score = roc_auc_score(None, None)
  ### END CODE ###
  print("Accuracy_score: {}, ROC_AUC_score: {}".format(acc_score, auc_score))

In [ ]:
print("Decision Tree")
# Initialize your decision tree classifier
### START CODE ###
dt_clf = None
### END CODE ###

dt_clf = fit_classifier(dt_clf)
evaluate_classifier(dt_clf, X_test, y_test)

b. (2.5 pts) Create a [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) of 25 classifiers (i.e, n_estimators=25) with random_state=14. Please use Decision Tree Classifier with random_state=14 as the base classifier. Report accuracy_score and roc_auc_score on the test data for this emsemble classifier.

In [ ]:
print("Bagging of Decesion Trees")
# Initialize your bagging classifier
### START CODE ###
bag_clf = None
### END CODE ###

bag_clf = fit_classifier(bag_clf)
evaluate_classifier(bag_clf, X_test, y_test)

c. (5 pts) In this question, you will fit a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) model on the training data for this classification task.

1. First, please find the best parameters (including *n_estimators*, *max_features* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the optimal parameters obtained by GridSearch.
2. Fit a model using the best parameters, and report the [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) on test data.

In [ ]:
def grid_search_for_classifier(clf, param_grid, X_train, y_train):
  # Grid search
  grid_search = GridSearchCV(clf, param_grid=param_grid)

  # Conduct grid search using the training set (1 line of code only)
  ### START CODE ###
  ### END CODE ###
  print(grid_search.best_params_)

  # Set the best paramters for your clf (1 line of code only)
  ### START CODE ###
  ### END CODE ###
  return clf

In [ ]:
def train_and_evaluate_classifier(clf, X_train, y_train, X_test, y_test):
  t0 = time()
  # Fit your classifier on the training set
  ### START CODE ###
  ### END CODE ###
  print("training time", round(time()-t0, 3), "s")

  t0 = time()
  y_pred = clf.predict(X_test)
  print("predict time", round(time()-t0, 3), "s")

  print("Confusion matrix: ")
  # Print the confusion matrix computed from the test set (1 line of code only)
  ### START CODE ###
  ### END CODE ###


  ### START CODE ###
  y_pred_proba = None
  acc_score = accuracy_score(None, None)
  auc_score = roc_auc_score(None, None)
  ### END CODE ###

  print("Accuracy: {}, AUC_ROC: {}".format(acc_score, auc_score))
  return clf

In [ ]:
param_grid = {"n_estimators": [1, 10, 50, 100],
              "max_features": [1, 5, 10, "auto"],
              "criterion": ['gini','entropy'],
              "random_state": [17]}

# Initialize your random forest classifier
### START CODE ###
rf_clf = None
### END CODE ###
rf_clf = grid_search_for_classifier(rf_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(rf_clf, X_train, y_train, X_test, y_test)

d. (10 pts) This time, let us use [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) and [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html) for the same task. For AdaBoost and XGBoost, please respectively find the best parameters (including *n_estimators, learning_rate*); fit your model using the best parameters, and report the confusion matrix and roc_auc_score on test data.

In [ ]:
param_grid = {"n_estimators": [10, 100],
          "learning_rate": [0.01, 0.1, 0.5],
          "random_state": [17]
          }

In [ ]:
# Initialize your AdaBoost classifier
### START CODE ###
ab_clf = None
### END CODE ###
ab_clf = grid_search_for_classifier(ab_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(ab_clf, X_train, y_train, X_test, y_test)

In [ ]:
# Initialize your XGBoost classifier
### START CODE ###
xgb_clf = None
### END CODE ###
xgb_clf = grid_search_for_classifier(xgb_clf, param_grid, X_train, y_train)
train_and_evaluate_classifier(xgb_clf, X_train, y_train, X_test, y_test)

f. (5 pts) Compare the performance of decision tree from part a) with the ensemble methods. Briefly explain which of the three ensemble methods performed better and why?

# Q3: CatBoost (10 points)

In this question you will learn about a boosting algorithm known as **CatBoost**. Please go through the two videos specified below to get a better understanding of the CatBoost algorithm and answer the questions that follow.

[Part-1](https://www.youtube.com/watch?v=KXOTSkPL2X4&ab_channel=StatQuestwithJoshStarmer)
[Part - 2](https://www.youtube.com/watch?v=3Bg2XRFOTzg&t=242s&ab_channel=StatQuestwithJoshStarmer)



a. **(5 points)** Briefly explain Ordered Target Encoding. What challenge does it try to address?

b. **(5 points)** Briefly describe the main advantages and disadvantages of CatBoost as compared to XGBoost.

# Q4: Convolutional Neural Network (20 points)
In this question, we will continue our exercise on the SVHN classification task from the previous homework, but this time we will be using Convolutional Neural Networks.

In [ ]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614))
        ])

train_dataset = torchvision.datasets.SVHN(root='.', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.SVHN(root='.', split='test', transform=transform, download=True)

In [ ]:
train_num = int(len(train_dataset) * 0.8)
val_num = len(train_dataset) - train_num
# Randomly split the training dataset into training dataset and validation dataset
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_num, val_num])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

a. (10 points) Build a convolutional neural network with the following sequential configuration. If not specified, please use the default setting of torch.nn.Conv2d. The output of the convolution layers will be fed into a fully-connected MLP. Then train the model with Adam optimizer (lr=1e-3) for 10 epochs. You should be able to achieve test accuracy of over 85%.



> Layer 1
*   2d convolution (# input channel=3, # output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 1
*   2d max pooling (kernel size=2)

> Layer 2
*   2d convolution (# output channel=16, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 2
*   2d max pooling (kernel size=2)

> Layer 3
*   2d convolution (# output channel=32, kernel size=3, padding=1)
*   2d batch normalization
*   Relu activation

> Pool 3
*   2d max pooling (kernel size=2)

References:

*   https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
*   https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html






In [ ]:
class CNN(nn.Module):
    def __init__(self, pool=True):
        super(CNN, self).__init__()
        self.pool = pool

        # Create convolutional layers
        ### START CODE ###
        self.layer1 = nn.Sequential()

        self.pool1 = None

        self.layer2 = nn.Sequential()

        self.pool2 = None

        self.layer3 = nn.Sequential()

        self.pool3 = None
        ### END CODE ###

        # Create fully connected layers (nn.Linear)
        if self.pool:
            self.mlp1 = nn.Linear(32*4*4, 50)
        else:
            self.mlp1 = nn.Linear(32*32*32, 50)

        self.mlp2 = nn.Linear(50, 50)
        self.mlp3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        if self.pool:
            x = self.pool1(x)

        x = self.layer2(x)
        if self.pool:
            x = self.pool2(x)

        x = self.layer3(x)
        if self.pool:
            x = self.pool3(x)
        x = x.reshape(x.shape[0], -1)

        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = self.mlp3(x)

        return x

In [ ]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Use the optimizer to perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    return avg_loss

@torch.no_grad()
def eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_num = 0
    for data, target in tqdm(loader):
        out = model(data)
        # Calculate loss based on model output and target
        loss = F.nll_loss(F.log_softmax(out, dim=1), target)

        # Get model's prediction
        pred = torch.argmax(out, dim=1)

        # Count number of correct predictions
        correct = accuracy_score(target, pred, normalize=False)

        total_correct += correct
        batch_size = len(target)
        total_loss += loss.item() * batch_size
        total_num += batch_size
    avg_loss = total_loss / total_num
    acc = total_correct / total_num
    return avg_loss, acc

In [ ]:
model1 = CNN(pool=True)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model1, train_loader, optimizer)
    val_loss, val_acc = eval(model1, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model1 = model1
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

In [ ]:
_, test_acc = eval(best_model1, test_loader)
print(f"Test accuracy: {np.round(test_acc, 3)}")

b. (5 points) Use torch-summary to print a summary of the model. The number of parameters should be less than the one of the MLP we trained in the previous homework. Why does it have less number of parameters but have higher accuracy?

Reference
*   https://pypi.org/project/torch-summary/


In [ ]:
!pip install torch-summary

c. (5 points) Train another CNN with the pool option set to False. What are the differences in terms of accuracy or computation caused by disabling max pooling? What are the effects of pooling operations in CNNs? (This might take some time. Watch a TV show while you're waiting for the results..)

In [ ]:
model2 = CNN(pool=False)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
best_acc = -np.inf
epochs = 10
for e in range(1, epochs + 1):
    train_loss = train(model2, train_loader, optimizer)
    val_loss, val_acc = eval(model2, val_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model2 = model2
    print(f"Epoch: {e} Train Loss: {train_loss} Val Loss: {val_loss} Val Acc: {val_acc}")

In [ ]:
_, test_acc = eval(best_model2, test_loader)
print(test_acc)